In [1]:
###################################################################################################
#Auxiliares
import os
import re
import pickle
import random
from time import time
import numpy as np
import tensorflow as tf
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Now using tensorflow 2.1.0, so no need to patch
# from tfdeterminism import patch
#patch()

seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
###################################################################################################
# utils
from distutils.version import LooseVersion
from tqdm import tqdm_notebook
from datetime import datetime
from time import time

import warnings
import pickle
import gc
import sys
from json import dumps
import itertools
import re

# Data
import spacy

# Viz
import matplotlib.pyplot as plt

# Machine Learning
import tensorflow.keras.backend as K
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Bidirectional, LSTM, Flatten, Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.callbacks.callbacks import EarlyStopping
from keras_self_attention import SeqSelfAttention
from keras_multi_head import MultiHead, MultiHeadAttention
###################################################################################################
#Dados
import pandas as pd
import matplotlib as pl

#preprocessing and transformation
from sklearn.preprocessing import normalize, MaxAbsScaler, MinMaxScaler, StandardScaler
from nltk.corpus import stopwords
#from nltk import word_tokenize, pos_tag

#Machine learning
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import LabelEncoder

#Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc, precision_recall_fscore_support, f1_score

seed = 42

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [4]:
#Variables
label='Atendimento'
exp = label + '-Unbalanced-Binary'
base_path = 'D:/03. Documentos/Mestrado/22032020 - Experimentos/05. Organizado/03. Datasets/'+exp
save_path = 'output'
data='resp-text'

# Models to be loaded
reglog_model_path = 'D:/03. Documentos/Mestrado/22032020 - Experimentos/05. Organizado/02. Notebooks/01. '+label+'/03. Resposta/output'
reglog_model_name = '2020_05_12_10_43_58_Atendimento_Resposta_Binary_Unbalanced.sav'
bilstm_mha_model_path = 'D:/Outputs_Mestrado/resultados_Atendimento/checkpoints_resposta_keras_BiLSTM_mha_binary_unbalanced'
bilstm_mha_model_name = '20200518_104250/model.h5'

x_train_file = 'X_train.csv'
y_train_file = 'y_train.csv'
x_test_file = 'X_test.csv'
y_test_file = 'y_test.csv'

#Load data
X_train = pd.read_csv(os.path.join(base_path, x_train_file), sep=';', encoding='utf-8')
y_train = pd.read_csv(os.path.join(base_path, y_train_file), sep=';', encoding='utf-8')
X_test = pd.read_csv(os.path.join(base_path, x_test_file), sep=';', encoding='utf-8')
y_test = pd.read_csv(os.path.join(base_path, y_test_file), sep=';', encoding='utf-8')

#Checking on data
print(X_test.columns)
print(X_test.shape)
print(y_test[label].value_counts())
print(y_test[label].value_counts())

Index(['pid', 'req-text', 'resp-text', '1funct-request', '2pronoun-request',
       '3ppron-request', '4i-request', '5we-request', '6you-request',
       '7shehe-request',
       ...
       '58home-response', '59money-response', '60relig-response',
       '61death-response', '62assent-response', '63nonfl-response',
       '64filler-response', 'Clareza', 'Atendimento', 'tempo_resposta'],
      dtype='object', length=134)
(13538, 134)
2    9338
0    4200
Name: Atendimento, dtype: int64
2    9338
0    4200
Name: Atendimento, dtype: int64


In [5]:
# load the RegLog model from disk
reglog_model = pickle.load(open(os.path.join(reglog_model_path, reglog_model_name), 'rb'))
bilstm_mha = load_model(os.path.join(bilstm_mha_model_path, bilstm_mha_model_name),
                        custom_objects={'MultiHeadAttention': MultiHeadAttention,
                                        'recall_m': recall_m,
                                        'precision_m': precision_m,
                                        'f1_m': f1_m})

In [6]:
max_num_words = bilstm_mha._layers[1]._trainable_weights[0].shape[0]
max_length = bilstm_mha.inputs[0].shape[1]

In [7]:
# PREDICT TRAIN
y_pred_reglog_train = reglog_model.predict(X_train[data])
y_pred_reglog_prob_train = reglog_model.predict_proba(X_train[data])

# PREDICT TEST
y_pred_reglog_test = reglog_model.predict(X_test[data])
y_pred_reglog_prob_test = reglog_model.predict_proba(X_test[data])

In [8]:
# Define tokenizer and fit train data
t = Tokenizer(num_words=max_num_words)
t.fit_on_texts(X_train[data].append(X_test[data]))
word_index = t.word_index
vocab_size = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))
    
def get_seqs(text):    
    sequences = t.texts_to_sequences(text)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded_sequences

# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return pd.get_dummies(y_train_enc), pd.get_dummies(y_test_enc)

# X and Y
label_train, label_test = prepare_targets(y_train[label].values, y_test[label].values)
num_labels = len(set(label_train))
input_train = get_seqs(X_train[data])
input_test = get_seqs(X_test[data])

Found 80284 unique tokens.


In [9]:
# PREDICT TRAIN
y_pred_bilstm_mha_prob_train = bilstm_mha.predict(input_train)
y_pred_bilstm_mha_train = np.argmax(y_pred_bilstm_mha_prob_train, axis=1)

# PREDICT TEST
y_pred_bilstm_mha_prob_test = bilstm_mha.predict(input_test)
y_pred_bilstm_mha_test = np.argmax(y_pred_bilstm_mha_prob_test, axis=1)

In [10]:
pipe = Pipeline([
    ('clf', LogisticRegression(random_state=seed, max_iter=1000, n_jobs=6, solver='lbfgs'))
])

parameters = {
        'clf__C': (0.001, 0.01, 0.1, 1, 10, 100, 1000),
}

grid_search_ensemble = GridSearchCV(pipe,
                               parameters,
                               cv=10,
                               scoring='f1_macro',
                               n_jobs=-1,
                               verbose=False)

In [18]:
df_prob_reglog_train = pd.DataFrame(y_pred_reglog_prob_train, columns=['reglog_0', 'reglog_2'])
df_prob_bilstm_mha_train = pd.DataFrame(y_pred_bilstm_mha_prob_train, columns=['bilstm_mha_0', 'bilstm_mha_2'])
df_y_train = pd.DataFrame(y_train[label], columns=[label]).reset_index(drop=True)
df_probs_train = pd.concat([df_prob_reglog_train, df_prob_bilstm_mha_train, y_train[label]], axis=1)

df_probs_train.head()

,reglog_0,reglog_2,bilstm_mha_0,bilstm_mha_2,Atendimento
0,0.610433,0.389567,0.476075,0.523925,2
1,0.748480,0.251520,0.694292,0.305708,2
2,0.417644,0.582356,0.036183,0.963817,2
3,0.173469,0.826531,0.045413,0.954587,2
4,0.198159,0.801841,0.095819,0.904181,2


In [19]:
print("Executando Gridsearch para Classe " + label + " - " + data)

# Time now
now = datetime.now().strftime("%Y%m%d_%H%M%S")
print(now)

t0 = time()
grid_search_ensemble.fit(df_probs_train.iloc[:, :-1], df_probs_train[label])
print("done in %0.3fs" % (time() - t0))
print("Best score: %0.3f" % grid_search_ensemble.best_score_)
print("Best parameters set:")
best_parameters_ensemble = grid_search_ensemble.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters_ensemble[param_name]))

Executando Gridsearch para Classe Atendimento - resp-text
20200607_105211
done in 1.758s
Best score: 0.810
Best parameters set:
	clf__C: 100


In [20]:
f_save = now + '_Ensemble_LR_' + exp + '_' + label + '.sav'
pickle.dump(grid_search_ensemble, open(f_save,'wb'))

In [21]:
df_prob_reglog_test = pd.DataFrame(y_pred_reglog_prob_test, columns=['reglog_0', 'reglog_2'])
df_prob_bilstm_mha_test = pd.DataFrame(y_pred_bilstm_mha_prob_test, columns=['bilstm_mha_0', 'bilstm_mha_2'])
df_y_test = pd.DataFrame(y_test[label], columns=[label]).reset_index(drop=True)
df_probs_test = pd.concat([df_prob_reglog_test, df_prob_bilstm_mha_test, y_test[label]], axis=1)

df_probs_test.head()

,reglog_0,reglog_2,bilstm_mha_0,bilstm_mha_2,Atendimento
0,0.270777,0.729223,0.046237,0.953763,0
1,0.260416,0.739584,0.185145,0.814855,0
2,0.353563,0.646437,0.586221,0.413779,0
3,0.385882,0.614118,0.125452,0.874548,0
4,0.777964,0.222036,0.908042,0.091958,0


In [23]:
y_pred_ensemble = grid_search_ensemble.predict(df_probs_test.iloc[:, :-1])

In [24]:
print('#'*50)
print('Report for TEST')
print('#'*50)
print(classification_report(df_probs_test.iloc[:,-1], y_pred_ensemble))

##################################################
Report for TEST
##################################################
              precision    recall  f1-score   support

           0       0.60      0.50      0.54      4200
           2       0.79      0.85      0.82      9338

    accuracy                           0.74     13538
   macro avg       0.69      0.67      0.68     13538
weighted avg       0.73      0.74      0.73     13538

